<a href="https://colab.research.google.com/github/hyunwook-lee/-/blob/master/PC_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
import numpy as np

In [21]:
spreadsheet = gc.open("PC_Rental")

In [22]:
worksheet = spreadsheet.sheet1  # 첫 번째 시트

# 데이터프레임으로 가져오기
df = get_as_dataframe(worksheet)
df = df.dropna(how='all')  # 빈 행 제거

In [5]:
df.columns = df.columns.str.strip()

In [6]:
# 📌 1. 예산 설정
budget = 6_736_186  # 위약금 + 유지 렌탈료의 합 최대값

# 📌 3. 데이터 전처리: '-' 처리 및 숫자형 변환
df.replace('-', 0, inplace=True)
df[['청구월단가', '위약금']] = df[['청구월단가', '위약금']].astype(float)

# 📌 4. 효율(렌탈료 절감 / 위약금) 계산
df['효율'] = df['청구월단가'] / df['위약금'].replace(0, float('inf'))

# 📌 5. 위약금이 0인 자산은 무조건 반납
free_return_df = df[df['위약금'] == 0]

# 📌 6. 위약금이 있는 자산들 중 효율이 높은 순으로 정렬
paid_df = df[df['위약금'] > 0].sort_values(by='효율', ascending=False).copy()

# 📌 7. 선택 가능한 자산 결정 (총 지출이 예산을 넘지 않도록)
selected = []
used_penalty = 0
total_rent_if_kept = df['청구월단가'].sum()

for idx, row in paid_df.iterrows():
    temp_penalty = row['위약금']
    # 현재까지 위약금 + 남은 자산 렌탈료 총합
    projected_total_cost = used_penalty + (total_rent_if_kept - row['청구월단가'])

    if projected_total_cost <= budget:
        selected.append(idx)
        used_penalty += temp_penalty
        total_rent_if_kept -= row['청구월단가']
    else:
        continue

# 📌 8. 반납 / 유지 자산 구분
return_df = pd.concat([free_return_df, paid_df.loc[selected]])
keep_df = df.drop(return_df.index)

# 📌 9. 총 지출 계산
total_penalty = return_df['위약금'].sum()
total_rent = keep_df['청구월단가'].sum()
total_spend = total_penalty + total_rent

# 📌 10. 결과 출력
print(f"✅ 반납할 자산 수: {len(return_df)}")
print(f"🖥️ 유지할 자산 수: {len(keep_df)}\n")
print(f"💰 위약금 총액: {total_penalty:,.0f}원")
print(f"💸 유지 자산 렌탈료 총액: {total_rent:,.0f}원")
print(f"📊 이번달 총 지출 예상 금액: {total_spend:,.0f}원")
print(f"🎯 예산 대비 잔여 금액: {budget - total_spend:,.0f}원")

✅ 반납할 자산 수: 121
🖥️ 유지할 자산 수: 144

💰 위약금 총액: 2,432,475원
💸 유지 자산 렌탈료 총액: 3,446,760원
📊 이번달 총 지출 예상 금액: 5,879,235원
🎯 예산 대비 잔여 금액: 856,951원


<ipython-input-6-d025a117b679>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', 0, inplace=True)


In [8]:
# 📌 11. 반납 자산 번호 목록 출력
print("\n📦 반납할 자산번호 목록:")
print(return_df['자산번호'].to_string(index=False))

# 📌 12. 유지 자산 번호 목록 출력
print("\n🖥️ 유지할 자산번호 목록:")
print(keep_df['자산번호'].to_string(index=False))


📦 반납할 자산번호 목록:
  11330887
  1CJ16301
  11769050
  11928552
  11936361
  11982945
  11992336
  11993381
  12005996
  12019797
  12055388
  12058957
  12068195
  120B7940
  120C9154
  120E3068
  12102020
  12102027
  12102383
  12120300
  12125373
  12134142
  12152402
  12159118
  12159119
  12159284
  12160368
  12162383
  12162387
  12183118
  12185378
  12189998
  121A3159
  121A6811
  121A7781
  121A7834
  121A7849
  12200139
  12200788
  12218410
  12224757
  12225071
  12221554
  12229067
  12229127
  12231940
  12232875
  12237916
  12244381
  12252533
  12394831
  12250174
  12257490
  12261963
  12164073
  12272684
  12258738
  12276696
  12277129
  12277130
  12277227
  12279029
  12282233
  122B3748
  12288121
  12288766
  12288822
  12291290
  12291690
  12251984
  12318364
  12296005
  12296862
  12296888
  12298017
  12298018
  12298020
  12298509
  122A0732
  122A0734
  122A5064
  122A0677
  122A2242
  12321228
  122B5089
  122B3876
  122B3908
  122D5043
  122B3831
  122

In [7]:
return_df

,자산번호,청구월단가,위약금,남은 렌탈 비용,효율
0,11330887,0.0,0.0,0,0.000000
1,1CJ16301,0.0,0.0,0,0.000000
2,11769050,0.0,0.0,0,0.000000
3,11928552,0.0,0.0,0,0.000000
4,11936361,0.0,0.0,0,0.000000
...,...,...,...,...,...
115,122D5315,27380.0,50200.0,191660,0.545418
119,122D5408,27380.0,50200.0,219040,0.545418
122,122D5420,27380.0,50200.0,219040,0.545418
118,12310059,37500.0,69000.0,300000,0.543478


In [40]:
from google.colab import files
files.download('선택된_PC_반납_자산번호.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
return_df.to_excel('선택된_PC_반납_자산번호.xlsx', index=False)

In [ ]:
with pd.ExcelWriter('/content/반납_및_유지_자산_리스트.xlsx') as writer:
    return_df.to_excel(writer, sheet_name='반납할 자산', index=False)
    keep_df.to_excel(writer, sheet_name='유지할 자산', index=False)

print("\n📁 결과 파일이 저장되었습니다: 반납_및_유지_자산_리스트.xlsx")


📁 결과 파일이 저장되었습니다: 반납_및_유지_자산_리스트.xlsx


In [19]:
import pandas as pd
import numpy as np
from itertools import combinations
from google.colab import files

# 📌 1. 예산 설정
budget = 6_736_186  # 위약금 + 유지 렌탈료 총합 상한


# 📌 3. 데이터 전처리
df.replace('-', 0, inplace=True)
df[['청구월단가', '위약금']] = df[['청구월단가', '위약금']].astype(float)
df['효율'] = df['청구월단가'] / df['위약금'].replace(0, np.inf)

# 📌 4. 위약금이 0인 자산은 무조건 반납
free_return_df = df[df['위약금'] == 0]
paid_df = df[df['위약금'] > 0].copy()

# 📌 5. 하이브리드 전략: 상위 효율 N개 자산 중 최적 조합 찾기
N = 20  # 너무 크게 하면 Colab 터질 수 있음
candidates = paid_df.sort_values(by='효율', ascending=False).head(N)

best_combo = []
max_saved_rent = 0
best_total_spend = 0

<ipython-input-19-681013f1a777>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', 0, inplace=True)


In [20]:
# 모든 조합 시도
for r in range(1, N + 1):
    for combo in combinations(candidates.index, r):
        temp_return_df = candidates.loc[list(combo)]
        penalty_sum = temp_return_df['위약금'].sum()
        saved_rent = temp_return_df['청구월단가'].sum()
        remaining_rent = df['청구월단가'].sum() - saved_rent
        total_spend = penalty_sum + remaining_rent

        if total_spend <= budget and saved_rent > max_saved_rent:
            best_combo = list(combo)
            max_saved_rent = saved_rent
            best_total_spend = total_spend

# 📌 6. 최종 반납 / 유지 자산 구분
return_df = pd.concat([free_return_df, paid_df.loc[best_combo]])
keep_df = df.drop(return_df.index)

# 📌 7. 지출 계산
total_penalty = return_df['위약금'].sum()
total_rent = keep_df['청구월단가'].sum()
total_spend = total_penalty + total_rent

# 📌 8. 결과 출력
print(f"✅ 반납 자산 수: {len(return_df)}")
print(f"🖥️ 유지 자산 수: {len(keep_df)}")
print(f"💰 총 위약금: {total_penalty:,.0f}원")
print(f"💸 유지 자산 렌탈료: {total_rent:,.0f}원")
print(f"📊 이번달 총 지출: {total_spend:,.0f}원")
print(f"🎯 예산 대비 잔여 금액: {budget - total_spend:,.0f}원")


✅ 반납 자산 수: 91
🖥️ 유지 자산 수: 174
💰 총 위약금: 737,350원
💸 유지 자산 렌탈료: 4,226,450원
📊 이번달 총 지출: 4,963,800원
🎯 예산 대비 잔여 금액: 1,772,386원


In [23]:
import pandas as pd
import numpy as np

# ✅ 예산 설정 (단위: 원)
budget = 6_736_186

df.replace('-', 0, inplace=True)
df[['청구월단가', '위약금']] = df[['청구월단가', '위약금']].astype(float)

# ✅ DP를 위한 세팅
unit = 1000  # 예산을 압축할 단위
budget_unit = int(budget // unit)
n = len(df)

dp = np.zeros((n + 1, budget_unit + 1))
keep = [[[] for _ in range(budget_unit + 1)] for _ in range(n + 1)]

# ✅ DP 알고리즘 수행
for i in range(1, n + 1):
    rent = df.loc[i - 1, '청구월단가']
    penalty = df.loc[i - 1, '위약금']
    penalty_unit = int(penalty // unit)

    for j in range(budget_unit + 1):
        # 반납하지 않는 경우
        dp[i][j] = dp[i - 1][j]
        keep[i][j] = keep[i - 1][j].copy()

        # 반납하는 경우 (예산 가능할 때만)
        if j >= penalty_unit:
            alt = dp[i - 1][j - penalty_unit] + rent
            if alt > dp[i][j]:
                dp[i][j] = alt
                keep[i][j] = keep[i - 1][j - penalty_unit].copy()
                keep[i][j].append(i - 1)

# ✅ 결과 계산
selected_indices = keep[n][budget_unit]
return_df = df.loc[selected_indices]
keep_df = df.drop(index=selected_indices)

total_penalty = return_df['위약금'].sum()
total_rent = keep_df['청구월단가'].sum()
total_spend = total_penalty + total_rent
remaining_budget = budget - total_spend
saved_rent = dp[n][budget_unit]

# ✅ 출력
print(f"✅ 반납 자산 수: {len(return_df)}")
print(f"🖥️ 유지 자산 수: {len(keep_df)}")
print(f"💰 위약금 총액: {total_penalty:,.0f}원")
print(f"💸 유지 렌탈료 총액: {total_rent:,.0f}원")
print(f"📊 이번달 총 지출 예상 금액: {total_spend:,.0f}원")
print(f"🎯 예산 대비 잔여 금액: {remaining_budget:,.0f}원")
print(f"📉 절감한 렌탈료 합계: {saved_rent:,.0f}원")

<ipython-input-23-4c5ec3cf26bb>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('-', 0, inplace=True)


✅ 반납 자산 수: 191
🖥️ 유지 자산 수: 74
💰 위약금 총액: 6,799,005원
💸 유지 렌탈료 총액: 1,823,270원
📊 이번달 총 지출 예상 금액: 8,622,275원
🎯 예산 대비 잔여 금액: -1,886,089원
📉 절감한 렌탈료 합계: 4,412,916원
